In [1]:
import os
import gc
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from pathlib import Path
from keras.utils import np_utils
from keras.models import load_model
from keras.metrics import Precision, Recall
from keras.preprocessing.text import one_hot
from sklearn.metrics import classification_report
from keras.callbacks import CSVLogger, EarlyStopping
from sklearn.model_selection import train_test_split

!pip install transformers
!pip install sentencepiece
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, TFAutoModel, PreTrainedTokenizerFast, \
    PreTrainedTokenizerBase, TFBertModel, TFXLNetModel, TFRobertaModel, BertTokenizer, XLNetTokenizer, RobertaTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 36.4 MB/s 
     |████████████████████████████████| 596 kB 95.0 MB/s 
     |████████████████████████████████| 6.6 MB 73.4 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 30.4 MB/s 


In [2]:
import nltk
import matplotlib.pyplot as plt

from re import sub
from pandas import DataFrame
from tabulate import tabulate
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from typing import Union, List, Tuple
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from pandas.io.parsers import TextFileReader
from collections import Counter

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('punkt')
__STOPWORDS = stopwords.words("english")
STEMMER = PorterStemmer()
LEMMATIZER = WordNetLemmatizer()


def process_data(text: str, do_stemming: bool = False, do_lemmas: bool = False, do_lowercase: bool = False) -> str:
    """
    @param text: The text to process. It will remove the money amounts, retweets, links,
                 hashtags, punctuation and it will lowercase all the words
    @param do_stemming: Steam words to have less in vocabulary if set to true
    @param do_lowercase: Lowercase input text if set to true
    @param do_lemmas: Lemmatize word to be at a dictionary representation if true
    @return: The new processed text as a list of words
    """
    text = sub(r" +", " ", text)
    text = sub(r"\S@\S\s?", "", text)
    text = sub(r"[0-9]+(?:.[0-9]+){3}", "", text)
    text = sub(r"\$\w*", "", text)
    text = sub(r"(RT)+", "", text)
    text = sub(r"(lt)+", "", text)
    text = sub(r"(gt)+", "", text)
    text = sub(r"@[a-zA-Z0-9\_]+", "", text)
    text = sub(r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)", "", text)
    text = sub(r"#", "", text)
    text = sub(r"\d+", "", text)
    text = sub("[^A-Za-z0-9]+", " ", text)  # also removes special characters since they are not alphanumeric
    if do_lowercase:
        text = text.lower()

    # tokenized_data = __TOKENIZER.tokenize(text)
    tokenized_data = word_tokenize(text)
    processed_data = []

    for word in tokenized_data:
        if word not in __STOPWORDS and word not in punctuation:
            if do_stemming and not do_lemmas:
                word = STEMMER.stem(word)
            elif do_lemmas and not do_stemming:
                word = LEMMATIZER.lemmatize(word)
            elif do_lemmas and do_stemming:
                raise ValueError(
                    f"Can't do both lemmatizing and stemming. Values for do_lemmas={do_lemmas} "
                    f"and do_stemming={do_stemming} cannot be true for both."
                )
            processed_data.append(word)

    return " ".join(processed_data)


def plot_train_data(csv_data: Union[TextFileReader, DataFrame], train_metric: str, validation_metric: str) -> None:
    plt.figure()
    plt.plot(csv_data[train_metric], color="blueviolet")
    plt.plot(csv_data[validation_metric], color="green")
    plt.title(f"{train_metric.capitalize()} over epochs")
    plt.legend(["train", "validation"])
    plt.xlabel("epoch")
    plt.ylabel(train_metric)
    plt.grid(visible=True)


def display_train_report_and_f1_score(csv_data: Union[TextFileReader, DataFrame]) -> None:
    headers = ["epoch", "accuracy", "loss", "precision", "recall"]
    train_rep = csv_data[csv_data["epoch"] == len(csv_data) - 1][headers]
    precision = train_rep["precision"].iloc[0]
    recall = train_rep["recall"].iloc[0]
    f1_score = 2 * precision * recall / (precision + recall)
    train_rep["f1_score"] = f1_score
    rep_data = train_rep.values.tolist()
    print('\033[92m')
    print("╒═════════════════╕")
    print("│ Training Report │")
    print(tabulate(rep_data, headers=[header.capitalize() for header in headers] + ["F1 Score"], tablefmt="fancy_grid"))


def display_readable_time(start_time: float, end_time: float) -> None:
    minutes = (end_time - start_time) / 60
    ss = (end_time - start_time) % 60
    hh = minutes / 60
    mm = minutes % 60
    print('\033[94m')
    print(f"Training time: %02d:%02d:%02d | {round(end_time - start_time, 2)} seconds" % (hh, mm, ss))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir train_local
# !wget /content/drive/MyDrive/dataset/abusive-language/vidgen-wassem-davidson-founta-zampieri.zip
!unzip /content/drive/MyDrive/dataset/abusive-language/vidgen-wassem-davidson-founta-zampieri.zip -d /content/train_local

Archive:  /content/drive/MyDrive/dataset/abusive-language/vidgen-wassem-davidson-founta-zampieri.zip
  inflating: /content/train_local/dynamically-hate-vidgen/Dynamically Generated Hate Dataset - annotation guidelines.pdf  
  inflating: /content/train_local/dynamically-hate-vidgen/Dynamically_Generated_Hate_Dataset_v0.2.2.csv  
  inflating: /content/train_local/dynamically-hate-vidgen/Dynamically_Generated_Hate_Dataset_v0.2.3.csv  
  inflating: /content/train_local/dynamically-hate-vidgen/README.md  
  inflating: /content/train_local/hatecheck-data/all_annotations.csv  
  inflating: /content/train_local/hatecheck-data/all_cases.csv  
  inflating: /content/train_local/hatecheck-data/LICENSE  
  inflating: /content/train_local/hatecheck-data/README.md  
  inflating: /content/train_local/hatecheck-data/template_placeholders.csv  
  inflating: /content/train_local/hatecheck-data/test_suite_annotations.csv  
  inflating: /content/train_local/hatecheck-data/test_suite_cases.csv  
  inflating

In [6]:
MAX_PADDING_LENGTH = 28

def encode_tweet(tweet: str, tokenizer, max_padding_length: int = MAX_PADDING_LENGTH):
    return tokenizer.encode_plus(
        # tweet,
        process_data(tweet, do_stemming=False, do_lemmas=False, do_lowercase=True),
        add_special_tokens=True,
        max_length=max_padding_length,
        truncation=True,
        padding="max_length",
        return_attention_mask=True
    )


def encode_tweets(tweets_text: list, tweets_labels, tokenizer,  max_padding_length: int = MAX_PADDING_LENGTH):
    if tweets_labels is not None:
        assert len(tweets_text) == len(tweets_labels), f"Features and labels must have the same lengths. " \
                                                       f"Your input ({len(tweets_text)}, {len(tweets_labels)})"

    input_ids = []
    token_type_ids = []
    attention_masks = []

    for tweet in tweets_text:
        tweet_ = encode_tweet(tweet, tokenizer, max_padding_length)
        input_ids.append(tweet_["input_ids"])
        token_type_ids.append(tweet_["token_type_ids"])
        attention_masks.append(tweet_["attention_mask"])

    if tweets_labels is not None:
        assert len(input_ids) == len(token_type_ids) == len(attention_masks) == len(tweets_labels), \
            "Arrays must have the same length."
        return np.array(input_ids), np.array(token_type_ids), np.array(attention_masks), np.array(tweets_labels)

    return np.array(input_ids), np.array(token_type_ids), np.array(attention_masks)


def encode_tweets_no_token(tweets_text: list, tweets_labels, tokenizer, max_padding_length: int = MAX_PADDING_LENGTH):
    if tweets_labels is not None:
        assert len(tweets_text) == len(tweets_labels), f"Features and labels must have the same lengths. " \
                                                       f"Your input ({len(tweets_text)}, {len(tweets_labels)})"

    input_ids = []
    attention_masks = []

    for tweet in tweets_text:
        tweet_ = encode_tweet(tweet, tokenizer, max_padding_length)
        input_ids.append(tweet_["input_ids"])
        attention_masks.append(tweet_["attention_mask"])

    if tweets_labels is not None:
        assert len(input_ids) == len(attention_masks) == len(tweets_labels), \
            "Arrays must have the same length."
        return np.array(input_ids), np.array(attention_masks), np.array(tweets_labels)

    return np.array(input_ids), np.array(attention_masks)


def generate_dict(input_ids, token_type_ids, attention_mask) -> dict:
    return {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask
    }


def generate_dict_no_token(input_ids, attention_mask) -> dict:
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }


def encode_one_hot_and_preprocess(texts: list, vocab_size: int) -> list:
    return [one_hot(
        # temp_text,
        process_data(temp_text, do_stemming=False, do_lemmas=True, do_lowercase=True),
        vocab_size,
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
        lower=False,
        split=' '
    ) for temp_text in texts]


def prepare_data_for_train(texts: list,  vocab_size: int, max_len: int = MAX_PADDING_LENGTH) -> np.ndarray:
    texts = encode_one_hot_and_preprocess(texts, vocab_size)
    texts = tf.keras.preprocessing.sequence.pad_sequences(texts, maxlen=max_len, padding="post")

    return np.array(texts)


def convert_hatecheck_to_numerical(labels: list) -> list:
    # Transform labels to numerical value
    for index, label in enumerate(labels):
        if label == "non-hateful":
            labels[index] = 0
        elif label == "hateful":
            labels[index] = 1
        else:
            raise ValueError("Class column must have only 'non-hateful', or 'hateful' values")

    return labels

def compute_hatecheck_predictions(model_preds, final_preds):
    for prediction, final_pred in zip(model_preds, final_preds):
        for index, pred_class in enumerate(prediction):
            if pred_class == max(prediction):
                prediction[index] = 1
            else:
                prediction[index] = 0

            if index == 0:
                if prediction[0] == 1.0:
                    final_pred[1] = 1.0
                else:
                    if prediction[1] == 1.0:
                        final_pred[1] = 1.0
                    else:
                        final_pred[1] = 0.0
            elif index == 1:
                if prediction[1] == 1.0:
                    final_pred[1] = 1.0
                else:
                    if prediction[0] == 1.0:
                        final_pred[1] = 1.0
                    else:
                        final_pred[1] = 0.0
            elif index == 2:
                if prediction[2] == 1.0:
                    final_pred[0] = 1.0
                else:
                    final_pred[0] = 0.0

    return model_preds, final_preds

def compute_hatecheck_predictions_wassem(model_preds, final_preds):
    for prediction, final_pred in zip(model_preds, final_preds):
        for index, pred_class in enumerate(prediction):
            if pred_class == max(prediction):
                prediction[index] = 1
            else:
                prediction[index] = 0

            if index == 1:
                if prediction[1] == 1.0:
                    final_pred[1] = 1.0
                else:
                    if prediction[2] == 1.0:
                        final_pred[1] = 1.0
                    else:
                        final_pred[1] = 0.0
            elif index == 2:
                if prediction[2] == 1.0:
                    final_pred[1] = 1.0
                else:
                    if prediction[1] == 1.0:
                        final_pred[1] = 1.0
                    else:
                        final_pred[1] = 0.0
            elif index == 0:
                if prediction[0] == 1.0:
                    final_pred[0] = 1.0
                else:
                    final_pred[0] = 0.0

    return model_preds, final_preds
  

def compute_hatecheck_predictions_founta(model_preds, final_preds):
    for prediction, final_pred in zip(model_preds, final_preds):
        for index, pred_class in enumerate(prediction):
            if pred_class == max(prediction):
                prediction[index] = 1
            else:
                prediction[index] = 0


            if index == 0:
                if prediction[0] == 1.0:
                    final_pred[0] = 1.0
                else:
                    if prediction[1] == 1.0:
                        final_pred[0] = 1.0
                    else:
                        final_pred[0] = 0.0
            elif index == 1:
                if prediction[1] == 1.0:
                    final_pred[0] = 1.0
                else:
                    if prediction[0] == 1.0:
                        final_pred[0] = 1.0
                    else:
                        final_pred[0] = 0.0
            elif index == 2:
                if prediction[2] == 1.0:
                    final_pred[1] = 1.0
                else:
                    if prediction[3] == 1.0:
                        final_pred[1] = 1.0
                    else:
                        final_pred[1] = 0.0
            elif index == 3:
                if prediction[3] == 1.0:
                    final_pred[1] = 1.0
                else:
                    if prediction[2] == 1.0:
                        final_pred[1] = 1.0
                    else:
                        final_pred[1] = 0.0
            

    return model_preds, final_preds

# Test all the implementations for Davidson dataset on HateCheck.

In [7]:
# TODO: Update these with the google stuff
HATE_CHECK_DIR = os.path.join("/content/train_local", "hatecheck-data")
DATASET_PATH = os.path.join(HATE_CHECK_DIR, "test_suite_cases.csv")
DAVIDSON_MODEL_LOGS_PATH = os.path.join("/content/drive/MyDrive/nlp-models/abusive-language/logs", "davidson")
DAVIDSON_MODEL_PATH = os.path.join("/content/drive/MyDrive/nlp-models/abusive-language/models", "davidson")

CNN_DAVIDSON_MODEL_PATH = os.path.join(DAVIDSON_MODEL_PATH, "cnn_model_lowercase_lemme.h5") 
BERT_DAVIDSON_MODEL_PATH = os.path.join(DAVIDSON_MODEL_PATH, "bert_large_davidson.h5") 
XLNET_DAVIDSON_MODEL_PATH = os.path.join(DAVIDSON_MODEL_PATH, "xlnet_large_davidson.h5") 
HATEBERT_DAVIDSON_MODEL_PATH = os.path.join(DAVIDSON_MODEL_PATH, "hatebert_large_davidson.h5") 
HATEXPLAIN_DAVIDSON_MODEL_PATH = os.path.join(DAVIDSON_MODEL_PATH, "hatexplain_large_davidson.h5") 
ROBERTA_DAVIDSON_MODEL_PATH = os.path.join(DAVIDSON_MODEL_PATH, "roberta_large_davidson.h5") 
BERTWEET_DAVIDSON_MODEL_PATH = os.path.join(DAVIDSON_MODEL_PATH, "bertweet_large_davidson.h5") 

BERT_TYPE = "bert-large-uncased"  # bert-large-uncased | bert-base-uncased
ROBERTA_TYPE = "roberta-large"  # roberta-base | roberta-large | roberta-large-mnli
HATEXPLAIN_TYPE = "Hate-speech-CNERG/bert-base-uncased-hatexplain"
HATEBERT_TYPE = "GroNLP/hateBERT"
BERTWEET_TYPE = "vinai/bertweet-large"  # vinai/bertweet-base | vinai/bertweet-large
XLNET_TYPE = "xlnet-large-cased"  # xlnet-large-cased | xlnet-base-caseded

# Clean: 1828 | No lowercase: 1316 | Lowercase: 1179 | Lowercase & Stemming: 1004 | Lowercase & Lemmas: 1114
hate_check_vocab = 1114
# Clean: 20 | No lowercase: 12 | Lowercase: 11 | Lowercase & Stemming: 11 | Lowercase & Lemmas: 11
hate_check_padding = 11

In [8]:
df = pd.read_csv(DATASET_PATH, delimiter=",")

hatecheck_texts = df["test_case"].tolist()
hatecheck_labels = convert_hatecheck_to_numerical(df["label_gold"].tolist())

hatecheck_labels = np_utils.to_categorical(hatecheck_labels)

In [9]:
cnn_davidson_data = prepare_data_for_train(hatecheck_texts, vocab_size=hate_check_vocab)
cnn_davidson_labels = np.array(hatecheck_labels)

cnn_davidson_model = load_model(CNN_DAVIDSON_MODEL_PATH)

cnn_davidson_predictions = cnn_davidson_model.predict(cnn_davidson_data)
cnn_predictions_for_hatecheck = np.ndarray((cnn_davidson_predictions.shape[0], 2))

cnn_davidson_preds, cnn_preds_for_hatecheck = compute_hatecheck_predictions(cnn_davidson_predictions, cnn_predictions_for_hatecheck)


print(cnn_davidson_preds)
print(cnn_preds_for_hatecheck)

print()
print(cnn_davidson_labels)
print(type(cnn_davidson_labels), type(cnn_davidson_preds))
print(f"\n{classification_report(cnn_davidson_labels, cnn_preds_for_hatecheck)}")
del cnn_davidson_model
gc.collect()

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.35      0.30      0.32      1165
           1       0.70      0.75      0.72      2563

   micro avg       0.61      0.61      0.61      3728
   macro avg       0.53      0.52      0.52      3728
weighted avg       0.59      0.61      0.60      3728
 samples avg       0.61      0.61      0.61      3728



657

In [10]:
bert_tokenizer = BertTokenizer.from_pretrained(BERT_TYPE, do_lower_case=True)
bert_test_ids, bert_test_tokens, bert_test_masks, bert_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=bert_tokenizer
    )
bert_test_data = generate_dict(bert_test_ids, bert_test_tokens, bert_test_masks)

bert_davidson_model = load_model(BERT_DAVIDSON_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})

bert_davidson_predictions = bert_davidson_model.predict(bert_test_data)
bert_predictions_for_hatecheck = np.ndarray((bert_davidson_predictions.shape[0], 2))

bert_davidson_preds, bert_preds_for_hatecheck = compute_hatecheck_predictions(bert_davidson_predictions, bert_predictions_for_hatecheck)


print(bert_davidson_preds)
print(bert_preds_for_hatecheck)

print()
print(bert_test_labels)
print(type(bert_test_labels), type(bert_davidson_preds))
print(f"\n{classification_report(bert_test_labels, bert_preds_for_hatecheck)}")
del bert_davidson_model
del bert_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.29      0.41      0.34      1165
           1       0.67      0.55      0.60      2563

   micro avg       0.51      0.51      0.51      3728
   macro avg       0.48      0.48      0.47      3728
weighted avg       0.55      0.51      0.52      3728
 samples avg       0.51      0.51      0.51      3728



745

In [11]:
xlnet_tokenizer = XLNetTokenizer.from_pretrained(XLNET_TYPE, do_lower_case=True)
xlnet_test_ids, xlnet_test_tokens, xlnet_test_masks, xlnet_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=xlnet_tokenizer
    )
xlnet_test_data = generate_dict(xlnet_test_ids, xlnet_test_tokens, xlnet_test_masks)
xlnet_davidson_model = load_model(XLNET_DAVIDSON_MODEL_PATH, custom_objects={'TFXLNetModel': TFXLNetModel})

xlnet_davidson_predictions = xlnet_davidson_model.predict(xlnet_test_data)
xlnet_predictions_for_hatecheck = np.ndarray((xlnet_davidson_predictions.shape[0], 2))

xlnet_davidson_preds, xlnet_preds_for_hatecheck = compute_hatecheck_predictions(xlnet_davidson_predictions, xlnet_predictions_for_hatecheck)


print(xlnet_davidson_preds)
print(xlnet_preds_for_hatecheck)

print()
print(xlnet_test_labels)
print(type(xlnet_test_labels), type(xlnet_davidson_preds))
print(f"\n{classification_report(xlnet_test_labels, xlnet_preds_for_hatecheck)}")


del xlnet_davidson_model
del xlnet_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/761 [00:00<?, ?B/s]

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.30      0.28      0.29      1165
           1       0.68      0.71      0.69      2563

   micro avg       0.57      0.57      0.57      3728
   macro avg       0.49      0.49      0.49      3728
weighted avg       0.56      0.57      0.57      3728
 samples avg       0.57      0.57      0.57      3728



40885

In [12]:
hatebert_tokenizer = AutoTokenizer.from_pretrained(HATEBERT_TYPE, do_lower_case=True)
hatebert_test_ids, hatebert_test_tokens, hatebert_test_masks, hatebert_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=hatebert_tokenizer
    )
hatebert_test_data = generate_dict(hatebert_test_ids, hatebert_test_tokens, hatebert_test_masks)
hatebert_davidson_model = load_model(HATEBERT_DAVIDSON_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})

hatebert_davidson_predictions = hatebert_davidson_model.predict(hatebert_test_data)
hatebert_predictions_for_hatecheck = np.ndarray((hatebert_davidson_predictions.shape[0], 2))

hatebert_davidson_preds, hatebert_preds_for_hatecheck = compute_hatecheck_predictions(hatebert_davidson_predictions, hatebert_predictions_for_hatecheck)


print(hatebert_davidson_preds)
print(hatebert_preds_for_hatecheck)

print()
print(hatebert_test_labels)
print(type(hatebert_test_labels), type(hatebert_davidson_preds))
print(f"\n{classification_report(hatebert_test_labels, hatebert_preds_for_hatecheck)}")



del hatebert_davidson_model
del hatebert_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/151 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.32      0.41      0.36      1165
           1       0.69      0.61      0.65      2563

   micro avg       0.55      0.55      0.55      3728
   macro avg       0.51      0.51      0.50      3728
weighted avg       0.58      0.55      0.56      3728
 samples avg       0.55      0.55      0.55      3728



50122

In [13]:
hatexplain_tokenizer = AutoTokenizer.from_pretrained(HATEXPLAIN_TYPE, do_lower_case=True)
hatexplain_test_ids, hatexplain_test_tokens, hatexplain_test_masks, hatexplain_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=hatexplain_tokenizer
    )
hatexplain_test_data = generate_dict(hatexplain_test_ids, hatexplain_test_tokens, hatexplain_test_masks)

hatexplain_davidson_model = load_model(HATEXPLAIN_DAVIDSON_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})


hatexplain_davidson_predictions = hatexplain_davidson_model.predict(hatexplain_test_data)
hatexplain_predictions_for_hatecheck = np.ndarray((hatexplain_davidson_predictions.shape[0], 2))

hatexplain_davidson_preds, hatexplain_preds_for_hatecheck = compute_hatecheck_predictions(hatexplain_davidson_predictions, hatexplain_predictions_for_hatecheck)


print(hatexplain_davidson_preds)
print(hatexplain_preds_for_hatecheck)

print()
print(hatexplain_test_labels)
print(type(hatexplain_test_labels), type(hatexplain_davidson_preds))
print(f"\n{classification_report(hatexplain_test_labels, hatexplain_preds_for_hatecheck)}")

del hatexplain_davidson_model
gc.collect()

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.29      0.46      0.36      1165
           1       0.67      0.49      0.57      2563

   micro avg       0.48      0.48      0.48      3728
   macro avg       0.48      0.48      0.46      3728
weighted avg       0.55      0.48      0.50      3728
 samples avg       0.48      0.48      0.48      3728



41537

In [14]:
roberta_tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_TYPE, do_lower_case=True)

roberta_test_ids, roberta_test_masks, roberta_test_labels = encode_tweets_no_token(
    tweets_text=hatecheck_texts,
    tweets_labels=hatecheck_labels,
    tokenizer=roberta_tokenizer
)
roberta_test_data = generate_dict_no_token(roberta_test_ids, roberta_test_masks)
roberta_davidson_model = load_model(ROBERTA_DAVIDSON_MODEL_PATH, custom_objects={'TFRobertaModel': TFRobertaModel})


roberta_davidson_predictions = roberta_davidson_model.predict(roberta_test_data)
roberta_predictions_for_hatecheck = np.ndarray((roberta_davidson_predictions.shape[0], 2))

roberta_davidson_preds, roberta_preds_for_hatecheck = compute_hatecheck_predictions(roberta_davidson_predictions, roberta_predictions_for_hatecheck)


print(roberta_davidson_preds)
print(roberta_preds_for_hatecheck)

print()
print(roberta_test_labels)
print(type(roberta_test_labels), type(roberta_davidson_preds))
print(f"\n{classification_report(roberta_test_labels, roberta_preds_for_hatecheck)}")


del roberta_davidson_model
del roberta_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.39      0.10      0.17      1165
           1       0.69      0.93      0.79      2563

   micro avg       0.67      0.67      0.67      3728
   macro avg       0.54      0.52      0.48      3728
weighted avg       0.60      0.67      0.60      3728
 samples avg       0.67      0.67      0.67      3728



49248

In [15]:
bertweet_tokenizer = AutoTokenizer.from_pretrained(BERTWEET_TYPE, do_lower_case=True)

bertweet_test_ids, bertweet_test_masks, bertweet_test_labels = encode_tweets_no_token(
    tweets_text=hatecheck_texts,
    tweets_labels=hatecheck_labels,
    tokenizer=bertweet_tokenizer
)
bertweet_test_data = generate_dict_no_token(bertweet_test_ids, bertweet_test_masks)

bertweet_davidson_model = load_model(BERTWEET_DAVIDSON_MODEL_PATH, custom_objects={'TFRobertaModel': TFRobertaModel})


bertweet_davidson_predictions = bertweet_davidson_model.predict(bertweet_test_data)
bertweet_predictions_for_hatecheck = np.ndarray((bertweet_davidson_predictions.shape[0], 2))

bertweet_davidson_preds, bertweet_preds_for_hatecheck = compute_hatecheck_predictions(bertweet_davidson_predictions, bertweet_predictions_for_hatecheck)


print(bertweet_davidson_preds)
print(bertweet_preds_for_hatecheck)

print()
print(bertweet_test_labels)
print(type(bertweet_test_labels), type(bertweet_davidson_preds))
print(f"\n{classification_report(bertweet_test_labels, bertweet_preds_for_hatecheck)}")

del bertweet_davidson_model
del bertweet_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/614 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.37      0.38      0.37      1165
           1       0.71      0.71      0.71      2563

   micro avg       0.60      0.60      0.60      3728
   macro avg       0.54      0.54      0.54      3728
weighted avg       0.61      0.60      0.60      3728
 samples avg       0.60      0.60      0.60      3728



80170

Macro F1-Score, we chosed this since we want to treat classes more equaly:

*   CNN: 0.53
*   BERT: 0.47
*   XLNet: 0.49
*   HateBERT: 0.5
*   Hatexplain: 0.46
*   RoBERTa: 0.48
*   BERTweet: 0.54


Micro F1-Score:

*   CNN: 0.57
*   BERT: 0.51
*   XLNet: 0.57
*   HateBERT: 0.55
*   Hatexplain: 0.48
*   RoBERTa: 0.67
*   BERTweet: 0.60

In [16]:
# CNN, BERT, XLNet, HateBERT, HateXplain, RoBERTa, BERTweet
weights = [0.53, 0.47, 0.49, 0.5, 0.46, 0.48, 0.54]
# weights = [0.57, 0.51, 0.57, 0.55, 0.48, 0.67, 0.60]
weights = np.array(weights) / sum(np.array(weights))
print(weights)

[0.15273775 0.13544669 0.14121037 0.14409222 0.13256484 0.13832853
 0.1556196 ]


In [17]:
davidson_predictions = cnn_davidson_predictions * weights[0] + bert_davidson_predictions * weights[1] + xlnet_davidson_predictions * weights[2] + \
                       hatebert_davidson_predictions * weights[3] + hatexplain_davidson_predictions * weights[4] + \
                       roberta_davidson_predictions * weights[5] + bertweet_davidson_predictions * weights[6]

davidson_predictions_for_hatecheck = np.ndarray((davidson_predictions.shape[0], 2))
davidson_preds, davidson_preds_for_hatecheck = compute_hatecheck_predictions(bert_davidson_predictions, bert_predictions_for_hatecheck)


print(davidson_preds)
print(davidson_preds_for_hatecheck)

print()
print(bert_test_labels)
print(type(bert_test_labels), type(davidson_preds))
print(f"\n{classification_report(bert_test_labels, davidson_preds_for_hatecheck)}")

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.29      0.41      0.34      1165
           1       0.67      0.55      0.60      2563

   micro avg       0.51      0.51      0.51      3728
   macro avg       0.48      0.48      0.47      3728
weighted avg       0.55      0.51      0.52      3728
 samples avg       0.51      0.51      0.51      3728



# Testing all the implementation for Wassem dataset on HateCheck

In [34]:
WASSEM_MODEL_LOGS_PATH = os.path.join("/content/drive/MyDrive/nlp-models/abusive-language/logs", "wassem")
WASSEM_MODEL_PATH = os.path.join("/content/drive/MyDrive/nlp-models/abusive-language/models", "wassem")

CNN_WASSEM_MODEL_PATH = os.path.join(WASSEM_MODEL_PATH, "cnn_model_lowercase_lemme.h5") 
BERT_WASSEM_MODEL_PATH = os.path.join(WASSEM_MODEL_PATH, "bert_large_wassem.h5") 
XLNET_WASSEM_MODEL_PATH = os.path.join(WASSEM_MODEL_PATH, "xlnet_large_wassem.h5") 
HATEBERT_WASSEM_MODEL_PATH = os.path.join(WASSEM_MODEL_PATH, "hatebert_large_wassem.h5") 
HATEXPLAIN_WASSEM_MODEL_PATH = os.path.join(WASSEM_MODEL_PATH, "hatexplain_large_wassem.h5") 
ROBERTA_WASSEM_MODEL_PATH = os.path.join(WASSEM_MODEL_PATH, "roberta_large_wassem.h5") 
BERTWEET_WASSEM_MODEL_PATH = os.path.join(WASSEM_MODEL_PATH, "bertweet_large_wassem.h5") 

BERT_TYPE = "bert-large-uncased"  # bert-large-uncased | bert-base-uncased
ROBERTA_TYPE = "roberta-large"  # roberta-base | roberta-large | roberta-large-mnli
HATEXPLAIN_TYPE = "Hate-speech-CNERG/bert-base-uncased-hatexplain"
HATEBERT_TYPE = "GroNLP/hateBERT"
BERTWEET_TYPE = "vinai/bertweet-large"  # vinai/bertweet-base | vinai/bertweet-large
XLNET_TYPE = "xlnet-large-cased"  # xlnet-large-cased | xlnet-base-caseded

# Clean: 1828 | No lowercase: 1316 | Lowercase: 1179 | Lowercase & Stemming: 1004 | Lowercase & Lemmas: 1114
hate_check_vocab = 1114
# Clean: 20 | No lowercase: 12 | Lowercase: 11 | Lowercase & Stemming: 11 | Lowercase & Lemmas: 11
hate_check_padding = 11

MAX_PADDING_LENGTH = 24 

In [26]:
cnn_wassem_data = prepare_data_for_train(hatecheck_texts, vocab_size=hate_check_vocab, max_len=MAX_PADDING_LENGTH)
cnn_wassem_labels = np.array(hatecheck_labels)

cnn_wassem_model = load_model(CNN_WASSEM_MODEL_PATH)

cnn_wassem_predictions = cnn_wassem_model.predict(cnn_wassem_data)
cnn_predictions_for_hatecheck = np.ndarray((cnn_wassem_predictions.shape[0], 2))

cnn_wassem_preds, cnn_preds_for_hatecheck = compute_hatecheck_predictions_wassem(cnn_wassem_predictions, cnn_predictions_for_hatecheck)


print(cnn_wassem_preds)
print(cnn_preds_for_hatecheck)

print()
print(cnn_wassem_labels)
print(type(cnn_wassem_labels), type(cnn_wassem_preds))
print(f"\n{classification_report(cnn_wassem_labels, cnn_preds_for_hatecheck)}")
del cnn_wassem_model
gc.collect()

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.31      0.86      0.46      1165
           1       0.69      0.14      0.24      2563

   micro avg       0.37      0.37      0.37      3728
   macro avg       0.50      0.50      0.35      3728
weighted avg       0.57      0.37      0.31      3728
 samples avg       0.37      0.37      0.37      3728



51274

In [27]:
bert_tokenizer = BertTokenizer.from_pretrained(BERT_TYPE, do_lower_case=True)
bert_test_ids, bert_test_tokens, bert_test_masks, bert_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=bert_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
bert_test_data = generate_dict(bert_test_ids, bert_test_tokens, bert_test_masks)

bert_wassem_model = load_model(BERT_WASSEM_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})

bert_wassem_predictions = bert_wassem_model.predict(bert_test_data)
bert_predictions_for_hatecheck = np.ndarray((bert_wassem_predictions.shape[0], 2))

bert_wassem_preds, bert_preds_for_hatecheck = compute_hatecheck_predictions_wassem(bert_wassem_predictions, bert_predictions_for_hatecheck)


print(bert_wassem_preds)
print(bert_preds_for_hatecheck)

print()
print(bert_test_labels)
print(type(bert_test_labels), type(bert_wassem_preds))
print(f"\n{classification_report(bert_test_labels, bert_preds_for_hatecheck)}")
del bert_wassem_model
del bert_tokenizer
gc.collect()

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.32      0.83      0.47      1165
           1       0.74      0.22      0.33      2563

   micro avg       0.41      0.41      0.41      3728
   macro avg       0.53      0.52      0.40      3728
weighted avg       0.61      0.41      0.37      3728
 samples avg       0.41      0.41      0.41      3728



50720

In [28]:
xlnet_tokenizer = XLNetTokenizer.from_pretrained(XLNET_TYPE, do_lower_case=True)
xlnet_test_ids, xlnet_test_tokens, xlnet_test_masks, xlnet_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=xlnet_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
xlnet_test_data = generate_dict(xlnet_test_ids, xlnet_test_tokens, xlnet_test_masks)
xlnet_wassem_model = load_model(XLNET_WASSEM_MODEL_PATH, custom_objects={'TFXLNetModel': TFXLNetModel})

xlnet_wassem_predictions = xlnet_wassem_model.predict(xlnet_test_data)
xlnet_predictions_for_hatecheck = np.ndarray((xlnet_wassem_predictions.shape[0], 2))

xlnet_wassem_preds, xlnet_preds_for_hatecheck = compute_hatecheck_predictions_wassem(xlnet_wassem_predictions, xlnet_predictions_for_hatecheck)


print(xlnet_wassem_preds)
print(xlnet_preds_for_hatecheck)

print()
print(xlnet_test_labels)
print(type(xlnet_test_labels), type(xlnet_wassem_preds))
print(f"\n{classification_report(xlnet_test_labels, xlnet_preds_for_hatecheck)}")


del xlnet_wassem_model
del xlnet_tokenizer
gc.collect()

[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.33      0.73      0.45      1165
           1       0.72      0.31      0.44      2563

   micro avg       0.45      0.45      0.45      3728
   macro avg       0.52      0.52      0.45      3728
weighted avg       0.60      0.45      0.44      3728
 samples avg       0.45      0.45      0.45      3728



71740

In [30]:
hatebert_tokenizer = AutoTokenizer.from_pretrained(HATEBERT_TYPE, do_lower_case=True)
hatebert_test_ids, hatebert_test_tokens, hatebert_test_masks, hatebert_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=hatebert_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
hatebert_test_data = generate_dict(hatebert_test_ids, hatebert_test_tokens, hatebert_test_masks)
hatebert_wassem_model = load_model(HATEBERT_WASSEM_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})

hatebert_wassem_predictions = hatebert_wassem_model.predict(hatebert_test_data)
hatebert_predictions_for_hatecheck = np.ndarray((hatebert_wassem_predictions.shape[0], 2))

hatebert_wassem_preds, hatebert_preds_for_hatecheck = compute_hatecheck_predictions_wassem(hatebert_wassem_predictions, hatebert_predictions_for_hatecheck)


print(hatebert_wassem_preds)
print(hatebert_preds_for_hatecheck)

print()
print(hatebert_test_labels)
print(type(hatebert_test_labels), type(hatebert_wassem_preds))
print(f"\n{classification_report(hatebert_test_labels, hatebert_preds_for_hatecheck)}")



del hatebert_wassem_model
del hatebert_tokenizer
gc.collect()

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.33      0.85      0.47      1165
           1       0.75      0.20      0.32      2563

   micro avg       0.41      0.41      0.41      3728
   macro avg       0.54      0.53      0.40      3728
weighted avg       0.62      0.41      0.37      3728
 samples avg       0.41      0.41      0.41      3728



77461

In [31]:
hatexplain_tokenizer = AutoTokenizer.from_pretrained(HATEXPLAIN_TYPE, do_lower_case=True)
hatexplain_test_ids, hatexplain_test_tokens, hatexplain_test_masks, hatexplain_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=hatexplain_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
hatexplain_test_data = generate_dict(hatexplain_test_ids, hatexplain_test_tokens, hatexplain_test_masks)

hatexplain_wassem_model = load_model(HATEXPLAIN_WASSEM_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})


hatexplain_wassem_predictions = hatexplain_wassem_model.predict(hatexplain_test_data)
hatexplain_predictions_for_hatecheck = np.ndarray((hatexplain_wassem_predictions.shape[0], 2))

hatexplain_wassem_preds, hatexplain_preds_for_hatecheck = compute_hatecheck_predictions_wassem(hatexplain_wassem_predictions, hatexplain_predictions_for_hatecheck)


print(hatexplain_wassem_preds)
print(hatexplain_preds_for_hatecheck)

print()
print(hatexplain_test_labels)
print(type(hatexplain_test_labels), type(hatexplain_wassem_preds))
print(f"\n{classification_report(hatexplain_test_labels, hatexplain_preds_for_hatecheck)}")

del hatexplain_wassem_model
gc.collect()

[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.32      0.89      0.47      1165
           1       0.75      0.15      0.25      2563

   micro avg       0.38      0.38      0.38      3728
   macro avg       0.54      0.52      0.36      3728
weighted avg       0.62      0.38      0.32      3728
 samples avg       0.38      0.38      0.38      3728



41537

In [32]:
roberta_tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_TYPE, do_lower_case=True)

roberta_test_ids, roberta_test_masks, roberta_test_labels = encode_tweets_no_token(
    tweets_text=hatecheck_texts,
    tweets_labels=hatecheck_labels,
    tokenizer=roberta_tokenizer,
    max_padding_length=MAX_PADDING_LENGTH
)
roberta_test_data = generate_dict_no_token(roberta_test_ids, roberta_test_masks)
roberta_wassem_model = load_model(ROBERTA_WASSEM_MODEL_PATH, custom_objects={'TFRobertaModel': TFRobertaModel})


roberta_wassem_predictions = roberta_wassem_model.predict(roberta_test_data)
roberta_predictions_for_hatecheck = np.ndarray((roberta_wassem_predictions.shape[0], 2))

roberta_wassem_preds, roberta_preds_for_hatecheck = compute_hatecheck_predictions_wassem(roberta_wassem_predictions, roberta_predictions_for_hatecheck)


print(roberta_wassem_preds)
print(roberta_preds_for_hatecheck)

print()
print(roberta_test_labels)
print(type(roberta_test_labels), type(roberta_wassem_preds))
print(f"\n{classification_report(roberta_test_labels, roberta_preds_for_hatecheck)}")


del roberta_wassem_model
del roberta_tokenizer
gc.collect()

[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.32      0.81      0.46      1165
           1       0.73      0.24      0.36      2563

   micro avg       0.41      0.41      0.41      3728
   macro avg       0.53      0.52      0.41      3728
weighted avg       0.60      0.41      0.39      3728
 samples avg       0.41      0.41      0.41      3728



65659

In [35]:
bertweet_tokenizer = AutoTokenizer.from_pretrained(BERTWEET_TYPE, do_lower_case=True)

bertweet_test_ids, bertweet_test_masks, bertweet_test_labels = encode_tweets_no_token(
    tweets_text=hatecheck_texts,
    tweets_labels=hatecheck_labels,
    tokenizer=bertweet_tokenizer,
    max_padding_length=MAX_PADDING_LENGTH
)
bertweet_test_data = generate_dict_no_token(bertweet_test_ids, bertweet_test_masks)

bertweet_wassem_model = load_model(BERTWEET_WASSEM_MODEL_PATH, custom_objects={'TFRobertaModel': TFRobertaModel})


bertweet_wassem_predictions = bertweet_wassem_model.predict(bertweet_test_data)
bertweet_predictions_for_hatecheck = np.ndarray((bertweet_wassem_predictions.shape[0], 2))

bertweet_wassem_preds, bertweet_preds_for_hatecheck = compute_hatecheck_predictions_wassem(bertweet_wassem_predictions, bertweet_predictions_for_hatecheck)


print(bertweet_wassem_preds)
print(bertweet_preds_for_hatecheck)

print()
print(bertweet_test_labels)
print(type(bertweet_test_labels), type(bertweet_wassem_preds))
print(f"\n{classification_report(bertweet_test_labels, bertweet_preds_for_hatecheck)}")

del bertweet_wassem_model
del bertweet_tokenizer
gc.collect()

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.31      0.96      0.47      1165
           1       0.62      0.03      0.05      2563

   micro avg       0.32      0.32      0.32      3728
   macro avg       0.47      0.50      0.26      3728
weighted avg       0.52      0.32      0.18      3728
 samples avg       0.32      0.32      0.32      3728



97417

Macro F1-Score, we chosed this since we want to treat classes more equaly:

*   CNN: 0.35
*   BERT: 0.40
*   XLNet: 0.45
*   HateBERT: 0.40
*   Hatexplain: 0.36
*   RoBERTa: 0.41
*   BERTweet: 0.26


Micro F1-Score:

*   CNN: 0.37
*   BERT: 0.41
*   XLNet: 0.45
*   HateBERT: 0.41
*   Hatexplain: 0.38
*   RoBERTa: 0.41
*   BERTweet: 0.32

In [36]:
# CNN, BERT, XLNet, HateBERT, HateXplain, RoBERTa, BERTweet
weights = [0.35, 0.40, 0.45, 0.40, 0.36, 0.41, 0.26]
# weights = [0.37, 0.41, 0.45, 0.41, 0.38, 0.41, 0.32]
weights = np.array(weights) / sum(np.array(weights))
print(weights)

[0.13307985 0.15209125 0.17110266 0.15209125 0.13688213 0.15589354
 0.09885932]


In [38]:
wassem_predictions = cnn_wassem_predictions * weights[0] + bert_wassem_predictions * weights[1] + xlnet_wassem_predictions * weights[2] + \
                       hatebert_wassem_predictions * weights[3] + hatexplain_wassem_predictions * weights[4] + \
                       roberta_wassem_predictions * weights[5] + bertweet_wassem_predictions * weights[6]

wassem_predictions_for_hatecheck = np.ndarray((wassem_predictions.shape[0], 2))
wassem_preds, wassem_preds_for_hatecheck = compute_hatecheck_predictions_wassem(bert_wassem_predictions, bert_predictions_for_hatecheck)


print(wassem_preds)
print(wassem_preds_for_hatecheck)

print()
print(bert_test_labels)
print(type(bert_test_labels), type(wassem_preds))
print(f"\n{classification_report(bert_test_labels, wassem_preds_for_hatecheck)}")

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.32      0.83      0.47      1165
           1       0.74      0.22      0.33      2563

   micro avg       0.41      0.41      0.41      3728
   macro avg       0.53      0.52      0.40      3728
weighted avg       0.61      0.41      0.37      3728
 samples avg       0.41      0.41      0.41      3728



# Test all the implementation for Vidgen dataset on HateCheck

In [51]:
VIDGEN_MODEL_LOGS_PATH = os.path.join("/content/drive/MyDrive/nlp-models/abusive-language/logs", "vidgen")
VIDGEN_MODEL_PATH = os.path.join("/content/drive/MyDrive/nlp-models/abusive-language/models", "vidgen")

CNN_VIDGEN_MODEL_PATH = os.path.join(VIDGEN_MODEL_PATH, "cnn_model_lowercase_lemme32.h5") 
BERT_VIDGEN_MODEL_PATH = os.path.join(VIDGEN_MODEL_PATH, "bert_base_vidgen.h5") 
XLNET_VIDGEN_MODEL_PATH = os.path.join(VIDGEN_MODEL_PATH, "xlnet_base_vidgen.h5") 
HATEBERT_VIDGEN_MODEL_PATH = os.path.join(VIDGEN_MODEL_PATH, "hatebert_large_vidgen.h5") 
HATEXPLAIN_VIDGEN_MODEL_PATH = os.path.join(VIDGEN_MODEL_PATH, "hatexplain_large_vidgen.h5") 
ROBERTA_VIDGEN_MODEL_PATH = os.path.join(VIDGEN_MODEL_PATH, "roberta_base_vidgen.h5") 
BERTWEET_VIDGEN_MODEL_PATH = os.path.join(VIDGEN_MODEL_PATH, "bertweet_base_vidgen.h5") 

BERT_TYPE = "bert-base-uncased"  # bert-large-uncased | bert-base-uncased
ROBERTA_TYPE = "roberta-base"  # roberta-base | roberta-large | roberta-large-mnli
HATEXPLAIN_TYPE = "Hate-speech-CNERG/bert-base-uncased-hatexplain"
HATEBERT_TYPE = "GroNLP/hateBERT"
BERTWEET_TYPE = "vinai/bertweet-base"  # vinai/bertweet-base | vinai/bertweet-large
XLNET_TYPE = "xlnet-base-cased"  # xlnet-large-cased | xlnet-base-caseded

# Clean: 1828 | No lowercase: 1316 | Lowercase: 1179 | Lowercase & Stemming: 1004 | Lowercase & Lemmas: 1114
hate_check_vocab = 1114
# Clean: 20 | No lowercase: 12 | Lowercase: 11 | Lowercase & Stemming: 11 | Lowercase & Lemmas: 11
hate_check_padding = 11

MAX_PADDING_LENGTH = 212 

In [49]:
import copy 

cnn_vidgen_data = prepare_data_for_train(hatecheck_texts, vocab_size=hate_check_vocab, max_len=MAX_PADDING_LENGTH)
cnn_vidgen_labels = np.array(hatecheck_labels)

cnn_vidgen_model = load_model(CNN_VIDGEN_MODEL_PATH)

cnn_vidgen_predictions = cnn_vidgen_model.predict(cnn_vidgen_data)
# cnn_predictions_for_hatecheck = np.ndarray((cnn_vidgen_predictions.shape[0], 2))
cnn_vidgen_predictions_ = copy.deepcopy(cnn_vidgen_predictions) 

# cnn_vidgen_preds, cnn_preds_for_hatecheck = compute_hatecheck_predictions_vidgen(cnn_vidgen_predictions, cnn_predictions_for_hatecheck)
for prediction in cnn_vidgen_predictions_:
    for index, pred_class in enumerate(prediction):
        if pred_class > 0.5:
            prediction[index] = 1
        else:
            prediction[index] = 0

cnn_vidgen_predictions_ = np_utils.to_categorical(cnn_vidgen_predictions_)

print(cnn_vidgen_predictions_)
# print(cnn_preds_for_hatecheck)

print()
print(cnn_vidgen_labels)
print(type(cnn_vidgen_labels), type(cnn_vidgen_predictions_))
print(f"\n{classification_report(cnn_vidgen_labels, cnn_vidgen_predictions_)}")
del cnn_vidgen_model
gc.collect()

[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.31      0.61      0.41      1165
           1       0.69      0.39      0.50      2563

   micro avg       0.46      0.46      0.46      3728
   macro avg       0.50      0.50      0.46      3728
weighted avg       0.57      0.46      0.47      3728
 samples avg       0.46      0.46      0.46      3728



2506

In [52]:
bert_tokenizer = BertTokenizer.from_pretrained(BERT_TYPE, do_lower_case=True)
bert_test_ids, bert_test_tokens, bert_test_masks, bert_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=bert_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
bert_test_data = generate_dict(bert_test_ids, bert_test_tokens, bert_test_masks)

bert_vidgen_model = load_model(BERT_VIDGEN_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})

bert_vidgen_predictions = bert_vidgen_model.predict(bert_test_data)
bert_vidgen_predictions_ = copy.deepcopy(bert_vidgen_predictions) 
# bert_predictions_for_hatecheck = np.ndarray((bert_vidgen_predictions.shape[0], 2))
# bert_vidgen_preds, bert_preds_for_hatecheck = compute_hatecheck_predictions_vidgen(bert_vidgen_predictions, bert_predictions_for_hatecheck)
for prediction in bert_vidgen_predictions_:
    for index, pred_class in enumerate(prediction):
        if pred_class > 0.5:
            prediction[index] = 1
        else:
            prediction[index] = 0

bert_vidgen_predictions_ = np_utils.to_categorical(bert_vidgen_predictions_)

print(bert_vidgen_predictions_)
# print(bert_preds_for_hatecheck)

print()
print(bert_test_labels)
print(type(bert_test_labels), type(bert_vidgen_predictions_))
print(f"\n{classification_report(bert_test_labels, bert_vidgen_predictions_)}")
del bert_vidgen_model
del bert_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.71      0.55      0.62      1165
           1       0.81      0.90      0.86      2563

   micro avg       0.79      0.79      0.79      3728
   macro avg       0.76      0.72      0.74      3728
weighted avg       0.78      0.79      0.78      3728
 samples avg       0.79      0.79      0.79      3728



27312

In [53]:
xlnet_tokenizer = XLNetTokenizer.from_pretrained(XLNET_TYPE, do_lower_case=True)
xlnet_test_ids, xlnet_test_tokens, xlnet_test_masks, xlnet_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=xlnet_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
xlnet_test_data = generate_dict(xlnet_test_ids, xlnet_test_tokens, xlnet_test_masks)
xlnet_vidgen_model = load_model(XLNET_VIDGEN_MODEL_PATH, custom_objects={'TFXLNetModel': TFXLNetModel})

xlnet_vidgen_predictions = xlnet_vidgen_model.predict(xlnet_test_data)
xlnet_vidgen_predictions_ = copy.deepcopy(xlnet_vidgen_predictions) 
# xlnet_predictions_for_hatecheck = np.ndarray((xlnet_vidgen_predictions.shape[0], 2))
# xlnet_vidgen_preds, xlnet_preds_for_hatecheck = compute_hatecheck_predictions_vidgen(xlnet_vidgen_predictions, xlnet_predictions_for_hatecheck)
for prediction in xlnet_vidgen_predictions_:
    for index, pred_class in enumerate(prediction):
        if pred_class > 0.5:
            prediction[index] = 1
        else:
            prediction[index] = 0

xlnet_vidgen_predictions_ = np_utils.to_categorical(xlnet_vidgen_predictions_)

print(xlnet_vidgen_predictions_)
print(xlnet_preds_for_hatecheck)

print()
print(xlnet_test_labels)
print(type(xlnet_test_labels), type(xlnet_vidgen_predictions_))
print(f"\n{classification_report(xlnet_test_labels, xlnet_vidgen_predictions_)}")


del xlnet_vidgen_model
del xlnet_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.71      0.45      0.55      1165
           1       0.79      0.92      0.85      2563

   micro avg       0.77      0.77      0.77      3728
   macro avg       0.75      0.69      0.70      3728
weighted avg       0.76      0.77      0.76      3728
 samples avg       0.77      0.77      0.77      3728



37480

In [54]:
hatebert_tokenizer = AutoTokenizer.from_pretrained(HATEBERT_TYPE, do_lower_case=True)
hatebert_test_ids, hatebert_test_tokens, hatebert_test_masks, hatebert_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=hatebert_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
hatebert_test_data = generate_dict(hatebert_test_ids, hatebert_test_tokens, hatebert_test_masks)
hatebert_vidgen_model = load_model(HATEBERT_VIDGEN_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})

hatebert_vidgen_predictions = hatebert_vidgen_model.predict(hatebert_test_data)
hatebert_vidgen_predictions_ = copy.deepcopy(hatebert_vidgen_predictions) 
# hatebert_predictions_for_hatecheck = np.ndarray((hatebert_vidgen_predictions.shape[0], 2))
# hatebert_vidgen_preds, hatebert_preds_for_hatecheck = compute_hatecheck_predictions_vidgen(hatebert_vidgen_predictions, hatebert_predictions_for_hatecheck)
for prediction in hatebert_vidgen_predictions_:
    for index, pred_class in enumerate(prediction):
        if pred_class > 0.5:
            prediction[index] = 1
        else:
            prediction[index] = 0

hatebert_vidgen_predictions_ = np_utils.to_categorical(hatebert_vidgen_predictions_)

print(hatebert_vidgen_predictions_)
# print(hatebert_preds_for_hatecheck)

print()
print(hatebert_test_labels)
print(type(hatebert_test_labels), type(hatebert_vidgen_predictions_))
print(f"\n{classification_report(hatebert_test_labels, hatebert_vidgen_predictions_)}")



del hatebert_vidgen_model
del hatebert_tokenizer
gc.collect()

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.76      0.58      0.66      1165
           1       0.83      0.92      0.87      2563

   micro avg       0.81      0.81      0.81      3728
   macro avg       0.80      0.75      0.77      3728
weighted avg       0.81      0.81      0.81      3728
 samples avg       0.81      0.81      0.81      3728



38410

In [56]:
hatexplain_tokenizer = AutoTokenizer.from_pretrained(HATEXPLAIN_TYPE, do_lower_case=True)
hatexplain_test_ids, hatexplain_test_tokens, hatexplain_test_masks, hatexplain_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=hatexplain_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
hatexplain_test_data = generate_dict(hatexplain_test_ids, hatexplain_test_tokens, hatexplain_test_masks)

hatexplain_vidgen_model = load_model(HATEXPLAIN_VIDGEN_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})


hatexplain_vidgen_predictions = hatexplain_vidgen_model.predict(hatexplain_test_data)
hatexplain_vidgen_predictions_ = copy.deepcopy(hatexplain_vidgen_predictions) 

# hatexplain_predictions_for_hatecheck = np.ndarray((hatexplain_vidgen_predictions.shape[0], 2))
# hatexplain_vidgen_preds, hatexplain_preds_for_hatecheck = compute_hatecheck_predictions_vidgen(hatexplain_vidgen_predictions, hatexplain_predictions_for_hatecheck)

for prediction in hatexplain_vidgen_predictions_:
    for index, pred_class in enumerate(prediction):
        if pred_class > 0.5:
            prediction[index] = 1
        else:
            prediction[index] = 0

hatexplain_vidgen_predictions_ = np_utils.to_categorical(hatexplain_vidgen_predictions_)

# print(hatexplain_vidgen_preds)
print(hatexplain_vidgen_predictions_)

print()
print(hatexplain_test_labels)
print(type(hatexplain_test_labels), type(hatexplain_vidgen_predictions_))
print(f"\n{classification_report(hatexplain_test_labels, hatexplain_vidgen_predictions_)}")

del hatexplain_vidgen_model
gc.collect()

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.77      0.60      0.67      1165
           1       0.83      0.92      0.87      2563

   micro avg       0.82      0.82      0.82      3728
   macro avg       0.80      0.76      0.77      3728
weighted avg       0.81      0.82      0.81      3728
 samples avg       0.82      0.82      0.82      3728



66609

In [57]:
roberta_tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_TYPE, do_lower_case=True)

roberta_test_ids, roberta_test_masks, roberta_test_labels = encode_tweets_no_token(
    tweets_text=hatecheck_texts,
    tweets_labels=hatecheck_labels,
    tokenizer=roberta_tokenizer,
    max_padding_length=MAX_PADDING_LENGTH
)
roberta_test_data = generate_dict_no_token(roberta_test_ids, roberta_test_masks)
roberta_vidgen_model = load_model(ROBERTA_VIDGEN_MODEL_PATH, custom_objects={'TFRobertaModel': TFRobertaModel})


roberta_vidgen_predictions = roberta_vidgen_model.predict(roberta_test_data)
roberta_vidgen_predictions_ = copy.deepcopy(roberta_vidgen_predictions) 

# roberta_predictions_for_hatecheck = np.ndarray((roberta_vidgen_predictions.shape[0], 2))
# roberta_vidgen_preds, roberta_preds_for_hatecheck = compute_hatecheck_predictions_vidgen(roberta_vidgen_predictions, roberta_predictions_for_hatecheck)

for prediction in roberta_vidgen_predictions_:
    for index, pred_class in enumerate(prediction):
        if pred_class > 0.5:
            prediction[index] = 1
        else:
            prediction[index] = 0

roberta_vidgen_predictions_ = np_utils.to_categorical(roberta_vidgen_predictions_)

print(roberta_vidgen_predictions_)

print()
print(roberta_test_labels)
print(type(roberta_test_labels), type(roberta_vidgen_predictions_))
print(f"\n{classification_report(roberta_test_labels, roberta_vidgen_predictions_)}")


del roberta_vidgen_model
del roberta_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.82      0.56      0.67      1165
           1       0.82      0.94      0.88      2563

   micro avg       0.82      0.82      0.82      3728
   macro avg       0.82      0.75      0.77      3728
weighted avg       0.82      0.82      0.81      3728
 samples avg       0.82      0.82      0.82      3728



41720

In [58]:
bertweet_tokenizer = AutoTokenizer.from_pretrained(BERTWEET_TYPE, do_lower_case=True)

bertweet_test_ids, bertweet_test_masks, bertweet_test_labels = encode_tweets_no_token(
    tweets_text=hatecheck_texts,
    tweets_labels=hatecheck_labels,
    tokenizer=bertweet_tokenizer,
    max_padding_length=MAX_PADDING_LENGTH
)
bertweet_test_data = generate_dict_no_token(bertweet_test_ids, bertweet_test_masks)

bertweet_vidgen_model = load_model(BERTWEET_VIDGEN_MODEL_PATH, custom_objects={'TFRobertaModel': TFRobertaModel})


bertweet_vidgen_predictions = bertweet_vidgen_model.predict(bertweet_test_data)
bertweet_vidgen_predictions_ = copy.deepcopy(bertweet_vidgen_predictions) 

# bertweet_predictions_for_hatecheck = np.ndarray((bertweet_vidgen_predictions.shape[0], 2))
# bertweet_vidgen_preds, bertweet_preds_for_hatecheck = compute_hatecheck_predictions_vidgen(bertweet_vidgen_predictions, bertweet_predictions_for_hatecheck)

for prediction in bertweet_vidgen_predictions_:
    for index, pred_class in enumerate(prediction):
        if pred_class > 0.5:
            prediction[index] = 1
        else:
            prediction[index] = 0

bertweet_vidgen_predictions_ = np_utils.to_categorical(bertweet_vidgen_predictions_)

print(bertweet_vidgen_predictions_)

print()
print(bertweet_test_labels)
print(type(bertweet_test_labels), type(bertweet_vidgen_predictions_))
print(f"\n{classification_report(bertweet_test_labels, bertweet_vidgen_predictions_)}")

del bertweet_vidgen_model
del bertweet_tokenizer
gc.collect()

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.91      0.60      0.72      1165
           1       0.84      0.97      0.90      2563

   micro avg       0.86      0.86      0.86      3728
   macro avg       0.88      0.79      0.81      3728
weighted avg       0.86      0.86      0.85      3728
 samples avg       0.86      0.86      0.86      3728



41549

Macro F1-Score, we chosed this since we want to treat classes more equaly:

*   CNN: 0.46
*   BERT: 0.74
*   XLNet: 0.76
*   HateBERT: 0.77
*   Hatexplain: 0.77
*   RoBERTa: 0.77
*   BERTweet: 0.81


Micro F1-Score:

*   CNN: 0.46
*   BERT: 0.79
*   XLNet: 0.77
*   HateBERT: 0.81
*   Hatexplain: 0.82
*   RoBERTa: 0.82
*   BERTweet: 0.86

In [59]:
# CNN, BERT, XLNet, HateBERT, HateXplain, RoBERTa, BERTweet
# weights = [0.46, 0.74, 0.76, 0.77, 0.77, 0.77, 0.81]
# weights = [0.46, 0.79, 0.77, 0.81, 0.82, 0.82, 0.86]
weights = [0.79, 0.77, 0.81, 0.82, 0.82, 0.86]

weights = np.array(weights) / sum(np.array(weights))
print(weights)

[0.13307985 0.15209125 0.17110266 0.15209125 0.13688213 0.15589354
 0.09885932]


In [64]:
# vidgen_predictions = cnn_vidgen_predictions * weights[0] + bert_vidgen_predictions * weights[1] + xlnet_vidgen_predictions * weights[2] + \
#                        hatebert_vidgen_predictions * weights[3] + hatexplain_vidgen_predictions * weights[4] + \
#                        roberta_vidgen_predictions * weights[5] + bertweet_vidgen_predictions * weights[6]

vidgen_predictions = bert_vidgen_predictions * weights[0] + xlnet_vidgen_predictions * weights[1] + hatebert_vidgen_predictions * weights[2] + \
                       hatexplain_vidgen_predictions * weights[3] + roberta_vidgen_predictions * weights[4] + \
                       bertweet_vidgen_predictions * weights[5]

vidgen_predictions_ = copy.deepcopy(vidgen_predictions) 

# vidgen_predictions_for_hatecheck = np.ndarray((vidgen_predictions.shape[0], 2))
# vidgen_preds, vidgen_preds_for_hatecheck = compute_hatecheck_predictions_vidgen(bert_vidgen_predictions, bert_predictions_for_hatecheck)

for prediction in vidgen_predictions_:
    for index, pred_class in enumerate(prediction):
        if pred_class > 0.5:
            prediction[index] = 1
        else:
            prediction[index] = 0

vidgen_predictions_ = np_utils.to_categorical(vidgen_predictions_)
print(vidgen_predictions_)

print()
print(bertweet_test_labels)
print(type(bertweet_test_labels), type(vidgen_predictions))
print(f"\n{classification_report(bertweet_test_labels, vidgen_predictions_)}")

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.84      0.61      0.70      1165
           1       0.84      0.95      0.89      2563

   micro avg       0.84      0.84      0.84      3728
   macro avg       0.84      0.78      0.80      3728
weighted avg       0.84      0.84      0.83      3728
 samples avg       0.84      0.84      0.84      3728



# Test all the implementation on the Founta dataset for HateCheck

In [67]:
FOUNTA_MODEL_LOGS_PATH = os.path.join("/content/drive/MyDrive/nlp-models/abusive-language/logs", "founta")
FOUNTA_MODEL_PATH = os.path.join("/content/drive/MyDrive/nlp-models/abusive-language/models", "founta")

CNN_FOUNTA_MODEL_PATH = os.path.join(FOUNTA_MODEL_PATH, "cnn_model_lowercase_lemme.h5") 
BERT_FOUNTA_MODEL_PATH = os.path.join(FOUNTA_MODEL_PATH, "bert_large_founta.h5") 
XLNET_FOUNTA_MODEL_PATH = os.path.join(FOUNTA_MODEL_PATH, "xlnet_large_founta.h5") 
HATEBERT_FOUNTA_MODEL_PATH = os.path.join(FOUNTA_MODEL_PATH, "hatebert_large_founta.h5") 
HATEXPLAIN_FOUNTA_MODEL_PATH = os.path.join(FOUNTA_MODEL_PATH, "hatexplain_large_founta.h5") 
ROBERTA_FOUNTA_MODEL_PATH = os.path.join(FOUNTA_MODEL_PATH, "roberta_large_founta.h5") 
BERTWEET_FOUNTA_MODEL_PATH = os.path.join(FOUNTA_MODEL_PATH, "bertweet_large_founta.h5") 

BERT_TYPE = "bert-large-uncased"  # bert-large-uncased | bert-base-uncased
ROBERTA_TYPE = "roberta-large"  # roberta-base | roberta-large | roberta-large-mnli
HATEXPLAIN_TYPE = "Hate-speech-CNERG/bert-base-uncased-hatexplain"
HATEBERT_TYPE = "GroNLP/hateBERT"
BERTWEET_TYPE = "vinai/bertweet-large"  # vinai/bertweet-base | vinai/bertweet-large
XLNET_TYPE = "xlnet-large-cased"  # xlnet-large-cased | xlnet-base-caseded

# Clean: 1828 | No lowercase: 1316 | Lowercase: 1179 | Lowercase & Stemming: 1004 | Lowercase & Lemmas: 1114
hate_check_vocab = 1114
# Clean: 20 | No lowercase: 12 | Lowercase: 11 | Lowercase & Stemming: 11 | Lowercase & Lemmas: 11
hate_check_padding = 11

MAX_PADDING_LENGTH = 30 

In [68]:
cnn_founta_data = prepare_data_for_train(hatecheck_texts, vocab_size=hate_check_vocab, max_len=MAX_PADDING_LENGTH)
cnn_founta_labels = np.array(hatecheck_labels)

cnn_founta_model = load_model(CNN_FOUNTA_MODEL_PATH)

cnn_founta_predictions = cnn_founta_model.predict(cnn_founta_data)
cnn_predictions_for_hatecheck = np.ndarray((cnn_founta_predictions.shape[0], 2))

cnn_founta_preds, cnn_preds_for_hatecheck = compute_hatecheck_predictions_founta(cnn_founta_predictions, cnn_predictions_for_hatecheck)

print(cnn_founta_preds)
print(cnn_preds_for_hatecheck)

print()
print(cnn_founta_labels)
print(type(cnn_founta_labels), type(cnn_preds_for_hatecheck))
print(f"\n{classification_report(cnn_founta_labels, cnn_preds_for_hatecheck)}")
del cnn_founta_model
gc.collect()

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.31      1.00      0.48      1165
           1       0.00      0.00      0.00      2563

   micro avg       0.31      0.31      0.31      3728
   macro avg       0.16      0.50      0.24      3728
weighted avg       0.10      0.31      0.15      3728
 samples avg       0.31      0.31      0.31      3728



18351

In [69]:
bert_tokenizer = BertTokenizer.from_pretrained(BERT_TYPE, do_lower_case=True)
bert_test_ids, bert_test_tokens, bert_test_masks, bert_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=bert_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
bert_test_data = generate_dict(bert_test_ids, bert_test_tokens, bert_test_masks)

bert_founta_model = load_model(BERT_FOUNTA_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})

bert_founta_predictions = bert_founta_model.predict(bert_test_data)
 
bert_predictions_for_hatecheck = np.ndarray((bert_founta_predictions.shape[0], 2))
bert_founta_preds, bert_preds_for_hatecheck = compute_hatecheck_predictions_founta(bert_founta_predictions, bert_predictions_for_hatecheck)



print(bert_preds_for_hatecheck)
# print(bert_preds_for_hatecheck)

print()
print(bert_test_labels)
print(type(bert_test_labels), type(bert_preds_for_hatecheck))
print(f"\n{classification_report(bert_test_labels, bert_preds_for_hatecheck)}")
del bert_founta_model
del bert_tokenizer
gc.collect()

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.35      0.45      0.39      1165
           1       0.71      0.61      0.66      2563

   micro avg       0.56      0.56      0.56      3728
   macro avg       0.53      0.53      0.52      3728
weighted avg       0.60      0.56      0.57      3728
 samples avg       0.56      0.56      0.56      3728



50362

In [70]:
xlnet_tokenizer = XLNetTokenizer.from_pretrained(XLNET_TYPE, do_lower_case=True)
xlnet_test_ids, xlnet_test_tokens, xlnet_test_masks, xlnet_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=xlnet_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
xlnet_test_data = generate_dict(xlnet_test_ids, xlnet_test_tokens, xlnet_test_masks)
xlnet_founta_model = load_model(XLNET_FOUNTA_MODEL_PATH, custom_objects={'TFXLNetModel': TFXLNetModel})

xlnet_founta_predictions = xlnet_founta_model.predict(xlnet_test_data)
xlnet_predictions_for_hatecheck = np.ndarray((xlnet_founta_predictions.shape[0], 2))
xlnet_founta_preds, xlnet_preds_for_hatecheck = compute_hatecheck_predictions_founta(xlnet_founta_predictions, xlnet_predictions_for_hatecheck)


print(xlnet_preds_for_hatecheck)

print()
print(xlnet_test_labels)
print(type(xlnet_test_labels), type(xlnet_preds_for_hatecheck))
print(f"\n{classification_report(xlnet_test_labels, xlnet_preds_for_hatecheck)}")


del xlnet_founta_model
del xlnet_tokenizer
gc.collect()

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.34      0.47      0.39      1165
           1       0.71      0.58      0.64      2563

   micro avg       0.55      0.55      0.55      3728
   macro avg       0.52      0.53      0.52      3728
weighted avg       0.59      0.55      0.56      3728
 samples avg       0.55      0.55      0.55      3728



71740

In [71]:
hatebert_tokenizer = AutoTokenizer.from_pretrained(HATEBERT_TYPE, do_lower_case=True)
hatebert_test_ids, hatebert_test_tokens, hatebert_test_masks, hatebert_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=hatebert_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
hatebert_test_data = generate_dict(hatebert_test_ids, hatebert_test_tokens, hatebert_test_masks)
hatebert_founta_model = load_model(HATEBERT_FOUNTA_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})

hatebert_founta_predictions = hatebert_founta_model.predict(hatebert_test_data)
hatebert_predictions_for_hatecheck = np.ndarray((hatebert_founta_predictions.shape[0], 2))
hatebert_founta_preds, hatebert_preds_for_hatecheck = compute_hatecheck_predictions_founta(hatebert_founta_predictions, hatebert_predictions_for_hatecheck)

print(hatebert_preds_for_hatecheck)

print()
print(hatebert_test_labels)
print(type(hatebert_test_labels), type(hatebert_preds_for_hatecheck))
print(f"\n{classification_report(hatebert_test_labels, hatebert_preds_for_hatecheck)}")

del hatebert_founta_model
del hatebert_tokenizer
gc.collect()

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.32      0.51      0.39      1165
           1       0.69      0.49      0.58      2563

   micro avg       0.50      0.50      0.50      3728
   macro avg       0.50      0.50      0.48      3728
weighted avg       0.57      0.50      0.52      3728
 samples avg       0.50      0.50      0.50      3728



50122

In [72]:
hatexplain_tokenizer = AutoTokenizer.from_pretrained(HATEXPLAIN_TYPE, do_lower_case=True)
hatexplain_test_ids, hatexplain_test_tokens, hatexplain_test_masks, hatexplain_test_labels = encode_tweets(
        tweets_text=hatecheck_texts,
        tweets_labels=hatecheck_labels,
        tokenizer=hatexplain_tokenizer,
        max_padding_length=MAX_PADDING_LENGTH
    )
hatexplain_test_data = generate_dict(hatexplain_test_ids, hatexplain_test_tokens, hatexplain_test_masks)

hatexplain_founta_model = load_model(HATEXPLAIN_FOUNTA_MODEL_PATH, custom_objects={'TFBertModel': TFBertModel})


hatexplain_founta_predictions = hatexplain_founta_model.predict(hatexplain_test_data)

hatexplain_predictions_for_hatecheck = np.ndarray((hatexplain_founta_predictions.shape[0], 2))
hatexplain_founta_preds, hatexplain_preds_for_hatecheck = compute_hatecheck_predictions_founta(hatexplain_founta_predictions, hatexplain_predictions_for_hatecheck)

print(hatexplain_preds_for_hatecheck)

print()
print(hatexplain_test_labels)
print(type(hatexplain_test_labels), type(hatexplain_preds_for_hatecheck))
print(f"\n{classification_report(hatexplain_test_labels, hatexplain_preds_for_hatecheck)}")

del hatexplain_founta_model
gc.collect()

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.34      0.51      0.41      1165
           1       0.71      0.55      0.62      2563

   micro avg       0.54      0.54      0.54      3728
   macro avg       0.53      0.53      0.51      3728
weighted avg       0.60      0.54      0.55      3728
 samples avg       0.54      0.54      0.54      3728



41537

In [73]:
roberta_tokenizer = RobertaTokenizer.from_pretrained(ROBERTA_TYPE, do_lower_case=True)

roberta_test_ids, roberta_test_masks, roberta_test_labels = encode_tweets_no_token(
    tweets_text=hatecheck_texts,
    tweets_labels=hatecheck_labels,
    tokenizer=roberta_tokenizer,
    max_padding_length=MAX_PADDING_LENGTH
)
roberta_test_data = generate_dict_no_token(roberta_test_ids, roberta_test_masks)
roberta_founta_model = load_model(ROBERTA_FOUNTA_MODEL_PATH, custom_objects={'TFRobertaModel': TFRobertaModel})


roberta_founta_predictions = roberta_founta_model.predict(roberta_test_data)

roberta_predictions_for_hatecheck = np.ndarray((roberta_founta_predictions.shape[0], 2))
roberta_founta_preds, roberta_preds_for_hatecheck = compute_hatecheck_predictions_founta(roberta_founta_predictions, roberta_predictions_for_hatecheck)

print(roberta_preds_for_hatecheck)

print()
print(roberta_test_labels)
print(type(roberta_test_labels), type(roberta_preds_for_hatecheck))
print(f"\n{classification_report(roberta_test_labels, roberta_preds_for_hatecheck)}")


del roberta_founta_model
del roberta_tokenizer
gc.collect()

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.37      0.42      0.40      1165
           1       0.72      0.67      0.70      2563

   micro avg       0.59      0.59      0.59      3728
   macro avg       0.55      0.55      0.55      3728
weighted avg       0.61      0.59      0.60      3728
 samples avg       0.59      0.59      0.59      3728



65659

In [74]:
bertweet_tokenizer = AutoTokenizer.from_pretrained(BERTWEET_TYPE, do_lower_case=True)

bertweet_test_ids, bertweet_test_masks, bertweet_test_labels = encode_tweets_no_token(
    tweets_text=hatecheck_texts,
    tweets_labels=hatecheck_labels,
    tokenizer=bertweet_tokenizer,
    max_padding_length=MAX_PADDING_LENGTH
)
bertweet_test_data = generate_dict_no_token(bertweet_test_ids, bertweet_test_masks)

bertweet_founta_model = load_model(BERTWEET_FOUNTA_MODEL_PATH, custom_objects={'TFRobertaModel': TFRobertaModel})


bertweet_founta_predictions = bertweet_founta_model.predict(bertweet_test_data)

bertweet_predictions_for_hatecheck = np.ndarray((bertweet_founta_predictions.shape[0], 2))
bertweet_founta_preds, bertweet_preds_for_hatecheck = compute_hatecheck_predictions_founta(bertweet_founta_predictions, bertweet_predictions_for_hatecheck)


print(bertweet_preds_for_hatecheck)

print()
print(bertweet_test_labels)
print(type(bertweet_test_labels), type(bertweet_preds_for_hatecheck))
print(f"\n{classification_report(bertweet_test_labels, bertweet_preds_for_hatecheck)}")

del bertweet_founta_model
del bertweet_tokenizer
gc.collect()

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.37      0.33      0.35      1165
           1       0.71      0.75      0.73      2563

   micro avg       0.62      0.62      0.62      3728
   macro avg       0.54      0.54      0.54      3728
weighted avg       0.61      0.62      0.61      3728
 samples avg       0.62      0.62      0.62      3728



80053

Macro F1-Score, we chosed this since we want to treat classes more equaly:

*   CNN: 0.24
*   BERT: 0.52
*   XLNet: 0.52
*   HateBERT: 0.48
*   Hatexplain: 0.51
*   RoBERTa: 0.55
*   BERTweet: 0.54


Micro F1-Score:

*   CNN: 0.31
*   BERT: 0.56
*   XLNet: 0.55
*   HateBERT: 0.50
*   Hatexplain: 0.54
*   RoBERTa: 0.59
*   BERTweet: 0.62

In [75]:
# CNN, BERT, XLNet, HateBERT, HateXplain, RoBERTa, BERTweet
# weights = [0.24, 0.52, 0.52, 0.48, 0.51, 0.55, 0.54]
# weights = [0.31, 0.56, 0.55, 0.50, 0.54, 0.59, 0.62]
weights = [0.56, 0.55, 0.50, 0.54, 0.59, 0.62]

weights = np.array(weights) / sum(np.array(weights))
print(weights)

[0.16666667 0.16369048 0.14880952 0.16071429 0.17559524 0.18452381]


In [77]:
# founta_predictions = cnn_founta_predictions * weights[0] + bert_founta_predictions * weights[1] + xlnet_founta_predictions * weights[2] + \
#                        hatebert_founta_predictions * weights[3] + hatexplain_founta_predictions * weights[4] + \
#                        roberta_founta_predictions * weights[5] + bertweet_founta_predictions * weights[6]

founta_predictions = bert_founta_predictions * weights[0] + xlnet_founta_predictions * weights[1] + hatebert_founta_predictions * weights[2] + \
                       hatexplain_founta_predictions * weights[3] + roberta_founta_predictions * weights[4] + \
                       bertweet_founta_predictions * weights[5]

founta_predictions_for_hatecheck = np.ndarray((founta_predictions.shape[0], 2))
founta_preds, founta_preds_for_hatecheck = compute_hatecheck_predictions_founta(bert_founta_predictions, bert_predictions_for_hatecheck)

print(founta_preds_for_hatecheck)

print()
print(bertweet_test_labels)
print(type(bertweet_test_labels), type(founta_preds_for_hatecheck))
print(f"\n{classification_report(bertweet_test_labels, founta_preds_for_hatecheck)}")

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>

              precision    recall  f1-score   support

           0       0.35      0.45      0.39      1165
           1       0.71      0.61      0.66      2563

   micro avg       0.56      0.56      0.56      3728
   macro avg       0.53      0.53      0.52      3728
weighted avg       0.60      0.56      0.57      3728
 samples avg       0.56      0.56      0.56      3728



# Voting between the datasets (2 * Wassem + 2 * Davidson + 2 * Founta + 3 * Vidgen) 

In [84]:
# print(davidson_preds_for_hatecheck)
# print()
# print(wassem_preds_for_hatecheck)
# print()
# print(vidgen_predictions_)
# print()
# print(founta_preds_for_hatecheck)

final_preds = (2 * davidson_preds_for_hatecheck + 2 * wassem_preds_for_hatecheck + 2 * founta_preds_for_hatecheck + 3 * vidgen_predictions_) / 9

print(final_preds)
for final_pred in final_preds:
    for index, pred_class in enumerate(final_pred):
        if pred_class == max(final_pred):
            final_pred[index] = 1
        else:
            final_pred[index] = 0 
print(final_preds)


print(f"\n{classification_report(np.array(hatecheck_labels), final_preds)}")


[[0.22222222 0.77777778]
 [0.22222222 0.77777778]
 [0.22222222 0.77777778]
 ...
 [0.66666667 0.33333333]
 [0.         1.        ]
 [0.66666667 0.33333333]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]

              precision    recall  f1-score   support

           0       0.49      0.65      0.56      1165
           1       0.81      0.69      0.75      2563

   micro avg       0.68      0.68      0.68      3728
   macro avg       0.65      0.67      0.65      3728
weighted avg       0.71      0.68      0.69      3728
 samples avg       0.68      0.68      0.68      3728

